In [1]:
import numpy as np
import time
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models.widgets import Slider, Select, Div, Button
from bokeh.events import ButtonClick
from bokeh.plotting import figure 
from bokeh.layouts import row, column, widgetbox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
output_notebook()

Loading BokehJS ...

In [2]:
timesteps = 120
samplingtime = 0.01
maxsetpoint = 10.0
timepoints = []
outputs = []
setpoints = []
errors = []

timepoints = np.zeros(timesteps)
outputs = np.zeros(timesteps)
setpoints = np.zeros(timesteps)
errors = np.zeros(timesteps)

In [3]:
class PID:
    def __init__(self, P, I, D):
        self.kp = P
        self.ki = I
        self.kd = D

    def initialize(self):
        # Initialize
        global timepoints 
        timepoints.fill(np.nan)
        global outputs 
        outputs.fill(np.nan)
        global setpoints 
        setpoints.fill(np.nan)
        global errors 
        errors.fill(np.nan)
        self.processval = 0.0  # Value obtained from measurement
        self.controlvar = 0.0  # Output from the PID controller
        self.setpoint = 0.0  # Desired value
        self.error = 0.0
        self.prop_term = 0.0
        self.inte_term = 0.0
        self.deri_term = 0.0
        self.lasterr = 0.0
        self.currenttime = time.time()
        self.lasttime = self.currenttime
    
    def controlLoop(self):
        # PID control logic
        self.error = self.setpoint - self.processval
        self.currenttime = time.time()
        dt = self.currenttime - self.lasttime
        if dt > samplingtime:
            self.prop_term = self.error
            self.inte_term = self.inte_term + self.error
            self.deri_term = self.error - self.lasterr
            self.controlvar = self.kp * self.prop_term + self.ki * self.inte_term + self.kd * self.deri_term
            self.lasterr = self.error
            self.lasttime = self.currenttime
        time.sleep(samplingtime + 0.01)

In [4]:
def modify_doc(doc):
    # Setup plots
    p1 = figure(plot_width=400, plot_height=300, x_axis_label='time', 
                y_range=[-1.5 * maxsetpoint, 1.5 * maxsetpoint], x_range=[0, timesteps * samplingtime], 
                title='PID controller input and output')
    p2 = figure(plot_width=400, plot_height=300, x_axis_label='time', y_axis_label='error',
                x_range=[0, timesteps * samplingtime], title='PID controller error')
    r1 = p1.line(timepoints, setpoints, line_color='cornflowerblue', legend='input',
                line_width=2)
    r2 = p1.line(timepoints, outputs, line_color='indianred', legend='output',
                line_width=2)
    r3 = p2.line(timepoints, errors, line_color='indigo',
                line_width=2)
    p1.legend.location = 'bottom_left'   
    p1.toolbar.logo = None
    p1.toolbar_location = None
    p2.toolbar.logo = None
    p2.toolbar_location = None
    
    # Setup widgets
    Kp = Slider(title='Kp', value=0.1, start=0.0, end=1.0, step=0.1)
    Ki = Slider(title='Ki', value=0.1, start=0.0, end=1.0, step=0.1)
    Kd = Slider(title='Kd', value=0.1, start=0.0, end=1.0, step=0.1)
    GNoise = Slider(title='Noise', value=0.0, start=0.0, end=1.0, step=0.2)
    InputFunction = Select(title='Input Function', value='step', options=['null', 'step', 'square', 'sine'])
    TextDisp = Div(text='''<b>Note:</b> Wait for the plots to stop updating before hitting Start.''')
    StartButton = Button(label='Start', button_type='success')
    
    
    def runPID(event):
        # Get current widget values
        kp = Kp.value
        ki = Ki.value
        kd = Kd.value
        InputFunc = InputFunction.value
        noise = GNoise.value
        
        pid = PID(kp, ki, kd)
        pid.initialize()
        delay = 20
        
        for nn in range(1, timesteps, 1):
            # Setpoints
            if nn > delay:
                if InputFunc == 'step': 
                    # step function
                    pid.setpoint = maxsetpoint 
                elif InputFunc == 'square':
                    # square wave
                    pid.setpoint = (maxsetpoint * 
                                    np.sign(np.sin(4 * np.pi * (nn - delay) / timesteps)) )
                elif InputFunc == 'sine':
                    # sine wave
                    pid.setpoint = maxsetpoint * np.sin(4 * np.pi * (nn - delay) / timesteps) 
                elif InputFunc == 'null':
                    pid.setpoint = 0.0
                # Controller output and measurement
                if noise == 0:
                    gauss_noise = 0
                else:
                    gauss_noise = noise * np.random.randn()  # Add measurement noise
                pid.processval = pid.processval + (pid.controlvar - (1.0 / nn)) + gauss_noise
            pid.controlLoop()

            global outputs
            outputs[nn] = pid.processval
            global timepoints
            timepoints[nn] = nn * samplingtime
            global setpoints
            setpoints[nn] = pid.setpoint
            global errors
            errors[nn] = pid.error

            # Update plots
            r1.data_source.data['x'] = timepoints
            r1.data_source.data['y'] = setpoints
            r2.data_source.data['x'] = timepoints
            r2.data_source.data['y'] = outputs
            r3.data_source.data['x'] = timepoints
            r3.data_source.data['y'] = errors
        
    # Setup callbacks
    StartButton.on_event(ButtonClick, runPID)
    
    # Setup layout and add to document
    wInputs = widgetbox(InputFunction, Kp, Ki, Kd, GNoise, TextDisp, StartButton)
    
    doc.add_root(column(wInputs, row(p1, p2, width=800)))

In [5]:
# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app, notebook_url="localhost:8888")